<a href="https://colab.research.google.com/github/zum-m/jouhougakugairon/blob/main/VGG16_FineTuning_kadai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.datasets import cifar10
import numpy as np

(x_train, y_train),(x_test, y_test) = cifar10.load_data()
#データの前処理
x_train=preprocess_input(x_train)
x_test=preprocess_input(x_test)

#教師ラベルをone-hot化
y_train_onehot = np.eye(10)[y_train]
y_train_onehot = np.squeeze(y_train_onehot)
y_test_onehot = np.eye(10)[y_test]
y_test_onehot = np.squeeze(y_test_onehot)

#学習済みのVGG16を特徴抽出器として使う
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(10,activation='softmax'))


model.summary()

#最後の方の層のみ再学習させる
# VGG16の最初の部分は重みを固定（frozen）

conv_base.trainable = True
for layer in conv_base.layers[:10]:
  layer.trainable = False


model.summary()

for i in model.layers:
    print(i.name, i.trainable)

early_stopping = EarlyStopping(monitor='loss',min_delta=0.01, patience=2)

#learning_rate は低めに設定
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-5), metrics=['accuracy'])
history = model.fit(x_train, y_train_onehot, batch_size=200, epochs=100, callbacks=[early_stopping])

#モデルの評価
loss, acc = model.evaluate(x_test,y_test_onehot)
print('Test loss:', loss, 'Test accuracy:', acc)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 activation_2 (Activation)   (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                32832     
                                                                 
 activation_3 (Activation)   (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 10)               

In [1]:
for i in model.layers:
    print(i.name, i.trainable)


NameError: ignored

---


VGG16_frozenとVGG16_FineTuningのテスト結果（テストデータに対する正答率）を記載し，その違いをもたらした要因について考察せよ．

⬇︎結果⬇︎

| VGG16_frozen | VGG16_FineTuning |
|---|---|
| 0.6220999956130981 | 0.8120999932289124 |

⬇︎考察⬇︎


---


ここに記述してください


VGG16_frozenとVGG16_FineTuningでは、frozenはVVGの事前学習モデルのパラメータを用いているが、CNNのアーキテクチャの層の後半はより抽象的な概念の理解に貢献するパラメータとなるなめ、frozenよりもVGG16のアーキテクチャの後半の層をチューニングしたVGG16_FineTuningの方が今回のデータのcifar10の出力に適した形に転移学習された状態になるから。

今回使用したVGG16の学習モデルはImageNetを使用しており、imagenetは1400万枚以上のカラー画像が収録されたもののため、今回の6万枚データのcifar10のみで1から学習させるよりは転移学習をさせた方が良い。時間的余裕があればimagenet以外のデータセットで学習したパラメータを持った事前学習モデルを用いて今回のタスク分類に適したモデルがないか試してみたい。